In [ ]:
%matplotlib inline
%matplotlib notebook
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from math import sin, cos, atan2, pi
from IPython.display import display, Math, Latex, Markdown, HTML
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
#Lidar data

Q = np.array([[4.00568,   1.01154,     0.0488547],
               [4.13862,  0.527872,    0.0469039],
               [4.11675, -1.17755,    -0.0615494],
               [4.06894, -1.19078,    -0.068076 ],
               [3.99535, -0.200271,   -0.0157148],
               [4.0255,   0.0652525,  -0.0162389],
               [3.97347,  0.0676774,  -0.0135881],
               [3.90745, -0.484574,    0.0426793],
               [13.9051,  0.0448338,  -0.336292],
               [13.9114,  0.0448533,  -0.336446],
               [13.9224,  0.0448875,  -0.336718]])  

   
#Radar data   
P = np.array([[3.97602,   1.04457,     0.273048 ],
               [4.02976,  0.531959,    0.352758 ],
               [4.01667, -1.22495,     0.0762318],
               [4.01538, -1.21231,     0.216883 ],
               [3.95006, -0.265133,    0.0912238],
               [3.9915,   0.0445874,   0.256736 ], 
               [3.87986, -0.0216695,   0.0243786],
               [3.84512, -0.518514,   -0.0243786],
               [13.602,  -0.189933,    0.998969],
               [13.6034, -0.189952,    0.979974], 
               [13.6034, -0.189952,    0.979974]]) 

PLx = Q[:,0]
PLy = Q[:,1]
PLz = Q[:,2]

PRx = P[:,0]
PRy = P[:,1]
PRz = P[:,2]

print(P.shape)
print(Q.shape)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(PLx, PLy, PLz, c='r', marker='o')
ax.scatter(PRx, PRy, PRz, c='b', marker='^')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
size = P.shape[1]
col=["red","blue","Maroon","yellow","olive","green"]

for i in range(size):
     
  #col.append(np.random.random(),np.random.random(),np.random.random())
  print(col[i])

In [ ]:
def plot_data(data_1, data_2, label_1, label_2, markersize_1=8, markersize_2=8):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    size = P.shape[1]
    ax.axis('auto')
    
    ####for i in range(size):
    col = (np.random.random(), np.random.random(), np.random.random())
    if data_1 is not None:
        x_p, y_p, z_p = data_1
        ax.plot(x_p,y_p,z_p,'o', color='blue',markersize=markersize_1,label=label_1)    
    if data_2 is not None: 
        x_q, y_q, z_q = data_2
        ax.plot(x_q,y_q,z_q,'o', color='red',markersize=markersize_2,label=label_2) 
        
    ax.legend()
    ax.set_xlabel('x [m]',fontsize=15)
    ax.set_ylabel('y [m]',fontsize=15)
    ax.set_zlabel('z [m]',fontsize=15)
    return ax, fig

In [ ]:
def plot_values(values, label):
    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)
    ax.plot(values, label=label)
    ax.legend()
    ax.grid(True)
    plt.show()

In [ ]:
print(P.T.shape)
P=P.T
Q=Q.T
print(P)
plot_data(P, Q, "Radar", "Lidar")
plt.show()

In [ ]:

def get_correspondence_indices(P, Q):
    """For each point in P find closest one in Q."""
    p_size = P.shape[1]
    q_size = Q.shape[1]
    correspondences = []
    for i in range(p_size):
        correspondences.append((i, i))
        for j in range(q_size):
            q_point = Q[:, i]
    return correspondences



In [ ]:
def draw_correspondeces(P, Q, correspondences, ax):
    label_added = False
    for i, j in correspondences:
        x = [P[0, i], Q[0, i]]
        y = [P[1, i], Q[1, i]]
        z = [P[2, i], Q[2, i]]
        if not label_added:
            ax.plot(x, y,z, color='grey', label='correspondences')
            label_added = True
        else:
            ax.plot(x, y, z, color='grey')
    ax.legend()


In [ ]:
def center_data(data, exclude_indices=[]):
    reduced_data = np.delete(data, exclude_indices, axis=1)
    center = np.array([reduced_data.mean(axis=1)]).T
    return center, (data - center)

center_of_P, P_centered = center_data(P)
center_of_Q, Q_centered = center_data(Q)
ax = plot_data(P_centered, Q_centered,
               label_1='Radar',
               label_2='Lidar')
plt.show()

In [ ]:
correspondences = get_correspondence_indices(P, Q)
ax,fig = plot_data(P, Q, "Radar", "Lidar")
draw_correspondeces(P, Q, correspondences, ax)
plt.show()







In [ ]:
def compute_cross_covariance(P, Q, correspondences, kernel=lambda diff: 1.0):
    cov = np.zeros((3, 3))
    exclude_indices = []
    for i, j in correspondences:
        p_point = P[:, [i]]
        q_point = Q[:, [i]]
        weight = kernel(p_point - q_point)
        if weight < 0.01: exclude_indices.append(i)
        cov += weight * p_point.dot(q_point.T)
    return cov, exclude_indices

cov, _ = compute_cross_covariance(P_centered, Q_centered, correspondences)
print(cov)

In [ ]:
U, S, V_T = np.linalg.svd(cov)
R_found = U.dot(V_T)  
t_found = center_of_Q - R_found.dot(center_of_P)

print("R_found =\n", R_found)
print("t_found =\n", t_found)



In [ ]:
from scipy.spatial.transform import Rotation   
r =  Rotation.from_matrix(R_found)
angles = r.as_euler("xyz",degrees=True)

print("Euler angles, [x,y,z] ", angles)



In [ ]:
P_corrected = R_found.dot(P) +t_found
ax = plot_data(P_corrected, Q, label_1='P corrected', label_2='Q')
plt.show()

correspondences = get_correspondence_indices(P_corrected, Q)


ax,fig = plot_data(P_corrected, Q, "Radar", "Lidar")
draw_correspondeces(P_corrected, Q, correspondences, ax)

plt.show()

In [ ]:
def plot_error(value1,value2,label1,label2):
    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)
    ax.plot(value1, label=label1, color='red',marker='o')
    ax.plot(value2, label=label2, color='blue',marker='o') 
 
    ax.legend()
    ax.grid(True)

    plt.xlabel("Images",fontsize=15,)
    plt.ylabel("Norm [m]",fontsize=15) 
    
    plt.show()

In [ ]:
import matplotlib.pyplot as pt

norm_values1 = []
error1=P-Q
for i in range(P.shape[1]):
    print("\n") 
    values1 = []
    for j in range(P.shape[0]):
        values1.append(error1[j][i])
        if(j==2):
            norm_values1.append(np.linalg.norm(np.array(values1)))
            
PQe=np.array(norm_values1)

norm_values2 = []
error2=P_corrected-Q
for i in range(P_corrected.shape[1]):
    print("\n") 
    values2 = []
    for j in range(P_corrected.shape[0]):
        values2.append(error2[j][i])
        if(j==2):
            norm_values2.append(np.linalg.norm(np.array(values2)))
            
P_correctedQe=np.array(norm_values2)

plot_error(PQe,P_correctedQe, label1="Norm_before_correction,     mean = 0.3390361025304447 ",label2="Norm_after_correction,        mean = 0.1300374920047055")

meanPQe  = np.mean(PQe)
#print(meanPQe)

meanP_correctedQe  = np.mean(P_correctedQe)
#print(meanP_correctedQe)

